In [1]:
# Import Dependencies
import pandas as pd

In [2]:
#create data frames from csv
schools_df = pd.read_csv('Resources/schools_complete.csv')
students_df = pd.read_csv('Resources/students_complete.csv')

In [3]:
# Merge data frames  
schools_full_df = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])
schools_full_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
## District Summary
#Calculate the total number of schools
num_schools = len(schools_df)

# Calculate the total number of students
num_students = len(students_df)

# Calculate the total budget
total_budget = schools_df['budget'].sum()

# Calculate the average math score 
math_score_ave = students_df['math_score'].mean()

# Calculate the average reading score
read_score_ave = students_df['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
math_pass_df = students_df.loc[students_df['math_score'] >= 70, :]
per_pass_math = len(math_pass_df)
per_pass_math = per_pass_math / num_students * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
per_pass_read = len(students_df.loc[students_df['reading_score'] >= 70, :])
per_pass_read = per_pass_read / num_students * 100

# Calculate the percentage of students who passed math **and** reading (% Overall Passing)
per_pass = len(math_pass_df.loc[math_pass_df['reading_score'] >= 70, :])
per_pass = per_pass / num_students * 100

# Create a dataframe to hold the above results
district_df = pd.DataFrame({'Total schools' : [num_schools],
             'Total students' : num_students,
             'Total budget' : total_budget,
             'Average math score' : math_score_ave,
             'Average reading score' : read_score_ave,
             '% Passing math' : per_pass_math,
             '% Passing reading' : per_pass_read,
             '% Passing both' : per_pass})

#Proper formatting
district_df['Total schools'] = district_df['Total schools'].map('{:,}'.format)
district_df['Total students'] = district_df['Total students'].map('{:,}'.format)
district_df['Total budget'] = district_df['Total budget'].map('${:,.2f}'.format)
district_df['Average math score'] = district_df['Average math score'].map('{:,.2f}'.format)
district_df['Average reading score'] = district_df['Average reading score'].map('{:,.2f}'.format)
district_df['% Passing math'] = district_df['% Passing math'].map('{:,.2f}%'.format)
district_df['% Passing reading'] = district_df['% Passing reading'].map('{:,.2f}%'.format)
district_df['% Passing both'] = district_df['% Passing both'].map('{:,.2f}%'.format)

district_df

,Total schools,Total students,Total budget,Average math score,Average reading score,% Passing math,% Passing reading,% Passing both
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


In [5]:
## School Summary

#School Name
indexes = schools_df['school_name'].values.tolist()

#School Type
school_type = pd.Series(schools_df['type'].values.tolist(),index=indexes)

#Total Students
total_students = students_df['school_name'].value_counts()

#Total School Budget
total_school_budget = pd.Series(schools_df['budget'].values.tolist(),index=indexes)

#Create DataFrame
summary_school = pd.DataFrame({'Total students' : total_students,
                              'Total budget' : total_school_budget,
                              'Type of school' : school_type})

#Per Student Budget
summary_school['Per student budget'] = summary_school['Total budget'] / summary_school['Total students']

#Average Math Score
average_math = schools_full_df.groupby('school_name')['math_score'].mean()
summary_school['Average math score'] = average_math

#Average Reading Score
average_read = schools_full_df.groupby('school_name')['reading_score'].mean()
summary_school['Average reading score'] = average_read

#% Passing Math (The percentage of students that passed math.)
math_per = schools_full_df.loc[schools_full_df['math_score']>= 70, :]
math_per.groupby('school_name')['math_score'].count()
summary_school['% Passing math'] = math_per.groupby('school_name')['math_score'].count() / summary_school['Total students'] * 100

#% Passing Reading (The percentage of students that passed reading.)
red_per = schools_full_df.loc[schools_full_df['reading_score']>= 70, :]
red_per = red_per.groupby('school_name')['reading_score'].count()
summary_school['% Passing reading'] = red_per / summary_school['Total students'] * 100

#% Overall Passing (The percentage of students that passed math **and** reading.)
math_per = schools_full_df.loc[schools_full_df['math_score']>= 70, :]
red_per = math_per.loc[math_per['reading_score']>= 70, :]
red_per = red_per.groupby('school_name')['reading_score'].count()
summary_school['% Passing both'] = red_per / summary_school['Total students'] * 100

#Proper formatting
summary_school = summary_school[['Type of school', 'Total students', 'Total budget', 'Per student budget',
                                'Average math score','Average reading score','% Passing math',
                                '% Passing reading','% Passing both']]

#summary_school['Total students'] = summary_school['Total students'].map('{:,}'.format)
#summary_school['Total budget'] = summary_school['Total budget'].map('${:,.2f}'.format)
#summary_school['Per student budget'] = summary_school['Per student budget'].map('${:,.2f}'.format)
#summary_school['Average math score'] = summary_school['Average math score'].map('{:,.2f}'.format)
#summary_school['Average reading score'] = summary_school['Average reading score'].map('{:,.2f}'.format)
#summary_school['% Passing math'] = summary_school['% Passing math'].map('{:,.2f}%'.format)
#summary_school['% Passing reading'] = summary_school['% Passing reading'].map('{:,.2f}%'.format)
#summary_school['% Passing both'] = summary_school['% Passing both'].map('{:,.2f}%'.format)


summary_school_print = summary_school.copy()
summary_school_print['Per student budget'] = summary_school_print['Per student budget'].map('${:,.2f}'.format)
summary_school_print['% Passing math'] = summary_school_print['% Passing math'].map('{:,.2f}%'.format)
summary_school_print['% Passing reading'] = summary_school_print['% Passing reading'].map('{:,.2f}%'.format)
summary_school_print['% Passing both'] = summary_school_print['% Passing both'].map('{:,.2f}%'.format)
summary_school_print['Average math score'] = summary_school_print['Average math score'].map('{:,.2f}'.format)
summary_school_print['Average reading score'] = summary_school_print['Average reading score'].map('{:,.2f}'.format)
summary_school_print['Total students'] = summary_school_print['Total students'].map('{:,}'.format)
summary_school_print['Total budget'] = summary_school_print['Total budget'].map('${:,.2f}'.format)



#summary_school_print
summary_school_print

,Type of school,Total students,Total budget,Per student budget,Average math score,Average reading score,% Passing math,% Passing reading,% Passing both
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


In [6]:
top_performance = summary_school_print.sort_values('% Passing both',ascending=False)
top_performance.head(5)



,Type of school,Total students,Total budget,Per student budget,Average math score,Average reading score,% Passing math,% Passing reading,% Passing both
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


In [7]:
bottom_performance = summary_school_print.sort_values('% Passing both')
bottom_performance.head(5)


,Type of school,Total students,Total budget,Per student budget,Average math score,Average reading score,% Passing math,% Passing reading,% Passing both
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


In [8]:
#Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

#Create a pandas series for each grade. Hint: use a conditional statement.
grades = list(students_df['grade'].unique())

tosortg = [int(grades[i][0:-2]) for i in range(len(grades))]
list2 = [[tosortg[i], grades[i]] for i in range(len(grades))]
list2.sort()
grades = [x[1] for x in list2]

dt = []
for grade in grades:
    v = students_df.loc[students_df['grade'] == grade]
    v2 = pd.Series(v['math_score'].values.tolist())
    v3 = pd.Series(v['school_name'].values.tolist())
    dt.append(pd.DataFrame({'School' : v3,
                     grade : v2}))  
#Group each series by school
for i in range(len(dt)):
    dt[i] = dt[i].groupby('School').mean()   
    
#Combine the series into a dataframe
merge_df = dt[0]
for i in range(1,len(dt)):
    merge_df = pd.merge(merge_df,dt[i],on='School')   

#Proper formatting   
for grade in grades:
    merge_df[grade] = merge_df[grade].map('{:.2f}'.format) 
    
    
    
merge_df





,9th,10th,11th,12th
School,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [9]:
#Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

#Create a pandas series for each grade. Hint: use a conditional statement.
grades = list(students_df['grade'].unique())

tosortg = [int(grades[i][0:-2]) for i in range(len(grade))]
list2 = [[tosortg[i], grades[i]] for i in range(len(grade))]
list2.sort()
grades = [x[1] for x in list2]

dt = []
for grade in grades:
    v = students_df.loc[students_df['grade'] == grade]
    v2 = pd.Series(v['reading_score'].values.tolist())
    v3 = pd.Series(v['school_name'].values.tolist())
    dt.append(pd.DataFrame({'School' : v3,
                     grade : v2}))  
#Group each series by school
for i in range(len(dt)):
    dt[i] = dt[i].groupby('School').mean()   
    
#Combine the series into a dataframe
merge_df = dt[0]
for i in range(1,len(dt)):
    merge_df = pd.merge(merge_df,dt[i],on='School')   

#Proper formatting   
for grade in grades:
    merge_df[grade] = merge_df[grade].map('{:.2f}'.format) 

    
merge_df


,9th,10th,11th,12th
School,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [15]:
#Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:

bins = [0, 585, 630, 645, 680]
sorting= [0,1,2,3]
# Create the names for the five bins
group_names = ['<$585', '$585-630','$630-645','$645-680']

summary_school_bins = summary_school
summary_school_bins['Spending ranges per student'] = pd.cut(summary_school_bins['Per student budget'], bins,
                                                            labels=group_names,include_lowest=True)
summary_school_bins['Sorting'] = pd.cut(summary_school_bins['Per student budget'], bins,
                                                            labels=sorting,include_lowest=True)
summary_school_bins = summary_school_bins[['Average math score','Average reading score','% Passing math','% Passing reading',
                                          '% Passing both','Spending ranges per student','Sorting']]
summary_school_bins = summary_school_bins.sort_values('Sorting')


indexes = summary_school_bins['Spending ranges per student'].values.tolist()

printingdf = pd.DataFrame({'Spending ranges per student': summary_school_bins['Spending ranges per student'].values.tolist(),
             'Average math score': summary_school_bins['Average math score'].values.tolist(),
             'Average reading score': summary_school_bins['Average reading score'].values.tolist(),
             '% Passing math': summary_school_bins['% Passing math'].values.tolist(),
             '% Passing reading': summary_school_bins['% Passing reading'].values.tolist(),
             '% Passing both': summary_school_bins['% Passing both'].values.tolist(),
                           'Sorting' : summary_school_bins['Sorting'].values.tolist()
             }).set_index(
    'Spending ranges per student').groupby('Spending ranges per student').mean().sort_values('Sorting')

del printingdf['Sorting']

#Proper formatting
printingdf['% Passing math'] = printingdf['% Passing math'].map('{:,.2f}%'.format)
printingdf['% Passing reading'] = printingdf['% Passing reading'].map('{:,.2f}%'.format)
printingdf['% Passing both'] = printingdf['% Passing both'].map('{:,.2f}%'.format)
printingdf['Average math score'] = printingdf['Average math score'].map('{:,.2f}'.format)
printingdf['Average reading score'] = printingdf['Average reading score'].map('{:,.2f}'.format)



printingdf


,Average math score,Average reading score,% Passing math,% Passing reading,% Passing both
Spending ranges per student,,,,,
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


In [23]:
bins2 = [0, 1000, 2000, 5000]

# Create the names for the five bins
group_names2 = ['Small (<1000)', 'Medium (1000-2000)','Large(2000-5000)']

summary_school_bins2 = pd.merge(summary_school,schools_df,how='left',left_index=True,right_on='school_name')
summary_school_bins2 = summary_school_bins2[['school_name','Total students','Total budget','Type of school',
                                             'Average math score','Average reading score',
                                         '% Passing math','% Passing reading','% Passing both','size']]

summary_school_bins2['School Size'] = pd.cut(summary_school_bins2['size'], bins2,
                                                            labels=group_names2,include_lowest=True)
summary_school_bins2 = summary_school_bins2[['Average math score','Average reading score','% Passing math','% Passing reading',
                                          '% Passing both','size','School Size']]



indexes = summary_school_bins2['School Size'].values.tolist()

printingdf2 = pd.DataFrame({'School Size':indexes,
             'Average math score':summary_school_bins2['Average math score'].values.tolist(),
             'Average reading score':summary_school_bins2['Average reading score'].values.tolist(),
             '% Passing math':summary_school_bins2['% Passing math'].values.tolist(),
             '% Passing reading':summary_school_bins2['% Passing reading'].values.tolist(),
             '% Passing both':summary_school_bins2['% Passing both'].values.tolist()}).set_index(
    'School Size').groupby('School Size').mean()

printingdf2['% Passing math'] = printingdf2['% Passing math'].map('{:,.2f}%'.format)
printingdf2['% Passing reading'] = printingdf2['% Passing reading'].map('{:,.2f}%'.format)
printingdf2['% Passing both'] = printingdf2['% Passing both'].map('{:,.2f}%'.format)
printingdf2['Average math score'] = printingdf2['Average math score'].map('{:,.2f}'.format)
printingdf2['Average reading score'] = printingdf2['Average reading score'].map('{:,.2f}'.format)
printingdf2




,Average math score,Average reading score,% Passing math,% Passing reading,% Passing both
School Size,,,,,
Large(2000-5000),77.75,81.34,69.96%,82.77%,58.29%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%


In [12]:
type_school = summary_school.groupby('Type of school')[['Average math score',
                                          'Average reading score','% Passing math',
                                          '% Passing reading','% Passing both']].mean()

printingdf2['% Passing math'] = printingdf2['% Passing math'].map('{:,.2f}%'.format)
printingdf2['% Passing reading'] = printingdf2['% Passing reading'].map('{:,.2f}%'.format)
printingdf2['% Passing both'] = printingdf2['% Passing both'].map('{:,.2f}%'.format)
printingdf2['Average math score'] = printingdf2['Average math score'].map('{:,.2f}'.format)
printingdf2['Average reading score'] = printingdf2['Average reading score'].map('{:,.2f}'.format)

,Average math score,Average reading score,% Passing math,% Passing reading,% Passing both
Type of school,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
